In [0]:
%reload_ext autoreload
%autoreload
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable #use to do backpropagation
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random


In [7]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
PATH = Path('/content/gdrive/My Drive')

In [9]:
wids = pd.read_csv(PATH/'train.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
wids.shape

(18255, 1235)

In [0]:
for col in wids.columns:
  if wids[col].isnull().sum() > 12000:
    wids.drop(col, axis = 1, inplace = True)


In [12]:
wids.shape

(18255, 422)

In [0]:
wids.to_csv(PATH/'train_422.csv')

In [0]:
train = pd.read_csv(PATH/'train_422.csv')

In [0]:
Y = train['is_female'].values.astype(np.float64)
X = train.drop(columns= ['is_female'])

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
for col in X.columns:
  if X.dtypes[col] == 'object':
    X[col] = X[col].fillna('NA')
  else:
    X[col] = X[col].fillna(0)
  le = LabelEncoder()
  X[col] = le.fit_transform(X[col])


In [0]:
for col in X.columns:
  X[col] = X[col].astype('category')

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 3)
X_train.head()

,Unnamed: 0,train_id,AA3,AA4,AA6,AA7,AA14,AA15,DG1,DG3,DG3A,DG4,DG5_1,DG5_2,DG5_3,DG5_4,DG5_5,DG5_6,DG5_7,DG5_8,DG5_9,DG5_10,DG5_11,DG5_96,DG6,DG8a,DG8b,DG8c,DG9a,DG9b,DG9c,DG10b,DG10c,DG11b,DL0,DL1,DL2,DL3,DL4_1,DL4_2,...,FB26_4,FB26_5,FB26_6,FB26_7,FB26_8,FB26_9,FB26_10,FB26_11,FB26_96,FB26_99,FB27_1,FB27_2,FB27_3,FB27_4,FB27_5,FB27_6,FB27_7,FB27_8,FB27_9,FB27_96,FB29_1,FB29_2,FB29_3,FB29_4,FB29_5,FB29_6,FB29_96,LN1A,LN1B,LN2_1,LN2_2,LN2_3,LN2_4,LN2_RIndLngBEOth,LN2_WIndLngBEOth,GN1,GN2,GN3,GN4,GN5
11369,11369,11369,1,10,2,424,482,252,62,2,3,4,0,1,1,1,1,0,1,0,1,0,1,0,0,2,1,0,1,0,0,0,0,0,0,10,0,0,1,1,...,2,2,1,2,2,2,2,2,2,2,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,2,0,0,0,38,38,0,1,1,1,1
1250,1250,1250,2,17,1,831,831,410,63,2,3,5,0,1,1,1,1,0,0,1,1,0,1,0,0,2,0,0,1,0,0,0,0,0,0,0,6,2,1,1,...,2,1,1,2,1,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,3,3,33,32,1,2,2,0,0
7527,7527,7527,1,8,0,301,310,166,53,2,3,0,0,1,1,1,1,0,0,1,1,0,1,0,1,3,1,3,2,0,0,1,3,1,1,3,2,2,1,1,...,2,2,1,2,1,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,0,0,0,0,38,38,2,1,1,1,1
13476,13476,13476,2,14,2,664,564,300,69,2,3,4,0,1,1,1,1,0,1,1,1,0,1,0,1,4,0,0,0,0,0,0,0,0,1,6,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,0,2,2,2,14,14,3,2,2,2,2
13406,13406,13406,0,12,3,587,111,76,58,2,3,2,0,1,1,1,1,0,0,1,1,0,1,0,0,2,1,0,1,0,0,1,0,1,0,0,30,2,1,1,...,2,2,2,2,1,2,2,2,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,3,3,14,14,1,0,2,2,2


In [19]:
emb_c = {n: len(col.cat.categories) for n, col in X.items() if len(col.cat.categories) > 2}
emb_c

{'AA14': 907,
 'AA15': 450,
 'AA3': 4,
 'AA4': 22,
 'AA6': 4,
 'AA7': 1050,
 'DG1': 79,
 'DG10b': 9,
 'DG10c': 8,
 'DG11b': 9,
 'DG3': 9,
 'DG3A': 8,
 'DG4': 12,
 'DG6': 9,
 'DG8a': 13,
 'DG8b': 13,
 'DG8c': 13,
 'DG9a': 12,
 'DG9b': 11,
 'DG9c': 8,
 'DL1': 12,
 'DL11': 15,
 'DL14': 24,
 'DL15': 4,
 'DL2': 33,
 'DL24': 7,
 'DL3': 3,
 'DL5': 25,
 'DL7': 3,
 'DL8': 342,
 'FB13': 25,
 'FB18': 5,
 'FB19': 11,
 'FB19B_1': 4,
 'FB19B_2': 4,
 'FB19B_3': 4,
 'FB19B_4': 4,
 'FB19B_5': 4,
 'FB19B_96': 4,
 'FB2': 3,
 'FB20': 16,
 'FB24': 17,
 'FB26_1': 3,
 'FB26_10': 3,
 'FB26_11': 3,
 'FB26_2': 3,
 'FB26_3': 3,
 'FB26_4': 3,
 'FB26_5': 3,
 'FB26_6': 3,
 'FB26_7': 3,
 'FB26_8': 3,
 'FB26_9': 3,
 'FB26_96': 3,
 'FB26_99': 3,
 'FF10_1': 3,
 'FF10_2': 3,
 'FF10_3': 3,
 'FF10_4': 3,
 'FF10_5': 3,
 'FF10_6': 3,
 'FF10_96': 3,
 'FF13': 8,
 'FF14_1': 3,
 'FF14_10': 3,
 'FF14_11': 3,
 'FF14_12': 3,
 'FF14_13': 3,
 'FF14_14': 3,
 'FF14_15': 3,
 'FF14_16': 3,
 'FF14_17': 3,
 'FF14_18': 3,
 'FF14_19': 3,
 '

In [0]:
emb_szs = [(c, min(50, (c + 1) // 2)) for _,c in emb_c.items()]

In [0]:
emb_cols = emb_c.keys()

In [0]:
class widsDataset(Dataset):
  def __init__(self, X, Y, emb_cols):
    X = X.copy()
    self.X1 = X.loc[:, emb_cols].copy().values.astype(np.int64)
    self.X2 = X.drop(columns = emb_cols).copy().values.astype(np.float32)
    self.y = Y

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return [self.X1[idx], self.X2[idx], self.y[idx]]


In [0]:
train_ds = widsDataset(X_train, Y_train, emb_cols)
valid_ds = widsDataset(X_val, Y_val, emb_cols)

In [0]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size = batch_size, shuffle = True)
valid_dl = DataLoader(valid_ds, batch_size = batch_size)

Model

In [0]:
class MixedInputModel(nn.Module):
  def __init__(self, emb_szs, n_cont):
    super().__init__()
    self.embs = nn.ModuleList([nn.Embedding(c, s) for c, s in emb_szs])
    n_emb = sum(e.embedding_dim for e in self.embs)
    self.n_emb, self.n_cont = n_emb, n_cont
    self.lin1 = nn.Linear(self.n_emb + self.n_cont, 100)
    self.lin2 = nn.Linear(100, 1)
    self.bn1 = nn.BatchNorm1d(self.n_cont)
    self.bn2 = nn.BatchNorm1d(100)
    self.emb_drop = nn.Dropout(0.5)
    self.drops = nn.Dropout(0.2)

  def forward(self, x_cat, x_cont):
    x = [e(x_cat[:,i]) for i, e in enumerate(self.embs)]
    x = torch.cat(x, 1)
    x = self.emb_drop(x)
    x2 = self.bn1(x_cont)
    x = torch.cat([x, x2], 1)
    x = F.relu(self.lin1(x))
    x = self.drops(x)
    x = self.bn2(x)
    x = self.lin2(x)
    return x

In [0]:
model = MixedInputModel(emb_szs, 172)

In [27]:
x1, x2, y = next(iter(train_dl))
print(x1.shape, x2.shape, y.shape)

torch.Size([5, 250]) torch.Size([5, 172]) torch.Size([5])


In [28]:
y = y.unsqueeze(1)
out = model(x1, x2)
out

tensor([[ 0.1707],
        [-0.1786],
        [-0.1172],
        [ 0.0375],
        [ 0.5010]], grad_fn=<AddmmBackward>)

In [29]:
out.shape

torch.Size([5, 1])

In [0]:
pred = (out > 0.0).float()

In [31]:
(pred == y).float().sum()

tensor(3.)

In [32]:
F.binary_cross_entropy_with_logits(out, y)

tensor(0.7367, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)

Training

In [0]:
def get_optimizer(model, lr = 0.01, wd = 0.0):
  parameters = filter(lambda p: p.requires_grad, model.parameters())
  optim = torch.optim.Adam(parameters, lr = lr, weight_decay= wd)
  return optim

In [0]:
def train_model(model, optim, train_dl = train_dl, verbose = False):
  model.train()
  total = 0
  sum_loss = 0
  for i, (x1, x2, y) in enumerate(train_dl):
    batch = y.shape[0]
    y = y.unsqueeze(1)
    out = model(x1, x2)
    loss = F.binary_cross_entropy_with_logits(out, y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    total += batch
    sum_loss += batch * (loss.item())
    if verbose: print(sum_loss/total)
  return sum_loss/total


In [0]:
def val_loss(model, valid_dl):
  model.eval()
  total = 0
  sum_loss = 0
  correct = 0
  for i, (x1, x2, y) in enumerate(valid_dl):
    batch = y.shape[0]
    y = y.unsqueeze(1)
    out = model(x1, x2)
    loss = F.binary_cross_entropy_with_logits(out, y)
    sum_loss += batch* (loss.item())
    total += batch
    pred = (out > 0).float()
    correct += (pred == y).float().sum().item()
  print("val_loss %.3f and accuracy %.3f" % (sum_loss/total, correct/total))
  return sum_loss/total, correct/total


In [0]:
from datetime import datetime

def train_loop(model, epochs, lr = 0.1, wd = 0.0):
  optim = get_optimizer(model, lr = lr, wd = wd)
  for i in range(epochs):
    loss = train_model(model, optim, train_dl)
    print("loss = ", loss)
    val_loss(model, valid_dl)

In [0]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size = batch_size)
valid_dl = DataLoader(valid_ds, batch_size = batch_size)

In [0]:
model = MixedInputModel(emb_szs, 172)

In [39]:
train_loop(model, epochs = 10, lr = 0.05, wd = 0.00001)

loss =  0.4399178168506565
val_loss 0.304 and accuracy 0.875
loss =  0.28318098602115993
val_loss 0.250 and accuracy 0.891
loss =  0.23981749311485412
val_loss 0.242 and accuracy 0.900
loss =  0.19719872880197492
val_loss 0.270 and accuracy 0.895
loss =  0.07611803467413786
val_loss 0.349 and accuracy 0.902
loss =  0.07789517251682328
val_loss 0.310 and accuracy 0.887
loss =  0.07244532254032976
val_loss 0.344 and accuracy 0.876
loss =  0.03200919267863558
val_loss 0.445 and accuracy 0.871
loss =  0.026930038575659513
val_loss 0.385 and accuracy 0.890
loss =  0.032678859803740926
val_loss 0.412 and accuracy 0.892
